# Exploratory Data Analysis (EDA)

Given that we have a DataFrame indexed by datetime, with columns for end time, unit name, PSR type, and quantity. The goal is to forecast the 'surplus' and develop an ARIMA model for that. Let's recap on the steps we need for this goal:

### Steps for Forecasting

#### 1. Data Preprocessing (Done in Data Wrangling)
- **Convert 'EndTime' to DateTime**: Convert the 'StartTime' column to a datetime object.
- **Set 'EndTime' as Index**: Set 'EndTime' as the DataFrame index for easier time series analysis.
- **Check for Missing Dates**: Ensure that all dates and times are represented. If there are missing intervals, decide how to handle them (e.g., filling with NaNs).
- **Aggregating Data**: Depending on the granularity you need, you might consider aggregating your data (e.g., daily average). In this case, the resample is done by hour. 

#### `2. Exploratory Data Analysis (In this notebook)`
- **Plotting the Series**: Visualize the 'surplus' over time to understand patterns, trends, and seasonality.
- **Seasonality and Trend Analysis**: Check if there is any visible seasonality or trend in the data which might influence the choice of the model.

#### 3. Model Selection
Given the nature of your data, you might consider the following models:
- **ARIMA/SARIMA**: If your data shows trends or autocorrelation. SARIMA is suitable if there is a clear seasonal pattern.
- **Prophet**: Handles daily data well, robust to missing data, and good with seasonality.
- **Machine Learning Approaches**: If there are other factors that can predict 'quantity', a model like Random Forest or Gradient Boosting might be useful.

#### 4. Model Training and Forecasting
- **Training**: Use data from January to April and October to December.
- **Forecasting**: Generate predictions for the missing months.
- **Hyperparameter Tuning**: Optimize model parameters for best performance.

#### 5. Model Evaluation
- **Performance Metrics**: Evaluate the model using appropriate metrics.
- **Cross-Validation**: If possible, use time series cross-validation.

In [1]:
# Imports
import pandas as pd
from ydata_profiling import ProfileReport
import matplotlib.pyplot as plt
#import seaborn as sns
#import numpy as np
#import statsmodels.api as sm
from statsmodels.tsa.arima.model import ARIMA

import statsmodels.api as sm
from statsmodels.tsa.stattools import adfuller#, acf, pacf
from statsmodels.graphics.tsaplots import plot_acf, plot_pacf
from statsmodels.tsa.seasonal import seasonal_decompose
from statsmodels.tsa.statespace.sarimax import SARIMAX

# Magic commands 
%matplotlib inline

Reading Data Wrangling output

In [2]:
# Parsing date strings, ignoring any timezone information and converting them to datetime objects
date_parser = lambda x: pd.to_datetime(x[:22])
eda_df = pd.read_csv("../2_data_wrangling/data_wrangling_output.csv", 
                     converters={'EndTime': date_parser}).set_index('EndTime')

Basic Data Inspection

In [3]:
eda_df.head()

,country_code,quantity_sum,Load,surplus
EndTime,,,,
2022-01-01 00:00:00+00:00,DE,146300.0,166143,-19843.0
2022-01-01 01:00:00+00:00,DE,140324.0,161923,-21599.0
2022-01-01 02:00:00+00:00,DE,134063.0,158256,-24193.0
2022-01-01 03:00:00+00:00,DE,128745.0,157353,-28608.0
2022-01-01 04:00:00+00:00,DE,120346.0,155306,-34960.0


In [4]:
eda_df.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 67889 entries, 2022-01-01 00:00:00+00:00 to 2022-12-31 23:00:00+00:00
Data columns (total 4 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   country_code  67889 non-null  object 
 1   quantity_sum  67889 non-null  float64
 2   Load          67889 non-null  int64  
 3   surplus       67889 non-null  float64
dtypes: float64(2), int64(1), object(1)
memory usage: 2.6+ MB


| Variable Name | Description                            | Non-Null Count | Data Type |
|---------------|----------------------------------------|----------------|-----------|
| **`Index`**     | Date and time (hourly)                 | 67889          | datetime  |
| `country_code`  | Identifier for the country             | 67889          | object    |
| `quantity_sum`  | Sum of quantities                      | 67889          | float64   |
| `Load`          | Load value                             | 67889          | int64     |
| `surplus`       | Energy surplus value                          | 67889          | float64   |

In [5]:
# JSON of country codes for mapping
countries_dict = {
    "SP": 0,  # Spain
    "UK": 1,  # United Kingdom
    "DE": 2,  # Germany
    "DK": 3,  # Denmark
    "HU": 5,  # Hungary
    "SE": 4,  # Sweden
    "IT": 6,  # Italy
    "PO": 7,  # Poland
    "NE": 8   # Netherlands
}

In [6]:
eda_df['country_code'].unique()

array(['DE', 'NE', 'DK', 'HU', 'SP', 'IT', 'SE', 'UK', 'PO'], dtype=object)

In [7]:
# Map the energy type codes to full names
eda_df['CountryLabel'] = eda_df['country_code'].map(countries_dict)

In [8]:
eda_df.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 67889 entries, 2022-01-01 00:00:00+00:00 to 2022-12-31 23:00:00+00:00
Data columns (total 5 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   country_code  67889 non-null  object 
 1   quantity_sum  67889 non-null  float64
 2   Load          67889 non-null  int64  
 3   surplus       67889 non-null  float64
 4   CountryLabel  67889 non-null  int64  
dtypes: float64(2), int64(2), object(1)
memory usage: 3.1+ MB


In [9]:
eda_df.head()

,country_code,quantity_sum,Load,surplus,CountryLabel
EndTime,,,,,
2022-01-01 00:00:00+00:00,DE,146300.0,166143,-19843.0,2
2022-01-01 01:00:00+00:00,DE,140324.0,161923,-21599.0,2
2022-01-01 02:00:00+00:00,DE,134063.0,158256,-24193.0,2
2022-01-01 03:00:00+00:00,DE,128745.0,157353,-28608.0,2
2022-01-01 04:00:00+00:00,DE,120346.0,155306,-34960.0,2


Basic Data Profiling

In [10]:
# Creating the data profiling
profile = ProfileReport(eda_df, title="Profiling Report")

profile.to_widgets() # Displaying the profile in the Jupyter notebook
profile.to_file("eda_df_profiling_output.html") # Saving the profiling in an html file

Summarize dataset:   0%|          | 0/5 [00:00<?, ?it/s]

c:\Users\pablo\.conda\envs\python311\Lib\site-packages\ydata_profiling\model\correlations.py:66: UserWarning: There was an attempt to calculate the auto correlation, but this failed.
To hide this warning, disable the calculation
(using `df.profile_report(correlations={"auto": {"calculate": False}})`
If this is problematic for your use case, please report this as an issue:
https://github.com/ydataai/ydata-profiling/issues
(include the error message: 'cannot reindex on an axis with duplicate labels')
  warnings.warn(


Generate report structure:   0%|          | 0/1 [00:00<?, ?it/s]

Render widgets:   0%|          | 0/1 [00:00<?, ?it/s]

Render HTML:   0%|          | 0/1 [00:00<?, ?it/s]

Export report to file:   0%|          | 0/1 [00:00<?, ?it/s]

In [11]:
# Finding duplicate rows
duplicates = eda_df.duplicated(keep=False) # 'keep=False' marks all duplicates as True

# Displaying duplicate rows
duplicate_rows = eda_df[duplicates]
print(duplicate_rows)

                          country_code  quantity_sum   Load  surplus  \
EndTime                                                                
2022-03-05 07:00:00+00:00           DK        1687.0   4225  -2538.0   
2022-05-03 17:00:00+00:00           DK        1278.0   4189  -2911.0   
2022-05-08 03:00:00+00:00           DK        2006.0   2763   -757.0   
2022-09-19 01:00:00+00:00           DK        2006.0   2763   -757.0   
2022-09-22 06:00:00+00:00           DK        1278.0   4189  -2911.0   
2022-09-28 04:00:00+00:00           DK        1235.0   3616  -2381.0   
2022-10-28 05:00:00+00:00           DK        1687.0   4225  -2538.0   
2022-11-13 13:00:00+00:00           DK        1235.0   3616  -2381.0   
2022-06-06 22:00:00+00:00           HU         612.0  15062 -14450.0   
2022-12-24 23:00:00+00:00           HU         612.0  15062 -14450.0   
2022-01-02 22:00:00+00:00           SP        7700.0  24535 -16835.0   
2022-04-28 04:00:00+00:00           SP        7700.0  24535 -168

Dropping Duplicates

In [12]:
eda_df = eda_df.drop_duplicates()

In [13]:
# Finding duplicate rows
duplicates = eda_df.duplicated(keep=False) # 'keep=False' marks all duplicates as True

# Displaying duplicate rows
duplicate_rows = eda_df[duplicates]
print(duplicate_rows)

Empty DataFrame
Columns: [country_code, quantity_sum, Load, surplus, CountryLabel]
Index: []


In [14]:
eda_df.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 67881 entries, 2022-01-01 00:00:00+00:00 to 2022-12-31 23:00:00+00:00
Data columns (total 5 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   country_code  67881 non-null  object 
 1   quantity_sum  67881 non-null  float64
 2   Load          67881 non-null  int64  
 3   surplus       67881 non-null  float64
 4   CountryLabel  67881 non-null  int64  
dtypes: float64(2), int64(2), object(1)
memory usage: 3.1+ MB


In [15]:
# Saving csv file for EDA and modeling
eda_df.to_csv('eda_output.csv')